# Pricing of Up-and-Out call option

### Import libraries

In [1]:
import numpy as np
from scipy.stats import norm
from scipy.stats import uniform
import random
import matplotlib.pyplot as plot


### Parameters

In [2]:
risk_free_rate = 0.08
T = 1
dT = 1/12

# barrier
L = 150

# firm related
V0 = 170    # arbitrarily selectd
D = 175     # debt
recover = 0.25     # recovery rate
S0 = 100    # current share price
sigma = 0.3     # share volatility
sigma_firm = 0.25    # firm value volatility
corr = 0.2    # correlation between share value and firm value

# strike
K = S0      # at-the-money option


### Utility functions

In [3]:
def terminal_shareprice(S_0, risk_free, sigma, Z, T):
    """Generates the terminal share price given some random normal values, Z"""
    return  S_0 * np.exp((risk_free - sigma**2 / 2)*T + sigma*np.sqrt(T)*Z)

def price_path(p_0, risk_free, sigma, Z, dT):
    """Generate the price path given intial price and some random normal values, Z"""
    return p_0 * np.exp(np.cumsum((risk_free - sigma**2/2)*dT + sigma*np.sqrt(dT)*Z, axis=0))
 
def call_payoff(share_path, K, r, T):
    """Calculating the payoff of up-and-out call option given a share price path."""
    if share_path.any() > L:
        return 0
    else:
        return np.maximum(share_path[-1] - K,0)
    

### Monte Carlo simulation

In [ ]:
np.random.seed(0)
corr_matrix = np.array([[1, corr], [corr, 1]])
sim_share_price = [None]*50
sim_firm_value = [None]*50
up_and_out_price = [None]*50
cva_estimates = [None]*50
call_price_adj = [None]*50

for i in range(1, 51):   # loop through sampling size
    z_norm = norm.rvs(size=[12, 2, i*1000])
    # correlated random numbers for share price and firm value
    z = np.array([np.matmul(np.linalg.cholesky(corr_matrix), x) for x in z_norm]) 
    # share price path
    share_price_path = np.array([price_path(S0, risk_free_rate, sigma, z_share, dT) for z_share in z[:,0,:].T])
    # firm value path
    firm_value_path  = np.array([price_path(V0, risk_free_rate, sigma_firm, z_firm, dT) for z_firm in z[:,1,:].T])
    # option payoff on each price path
    up_and_out_payoff = np.array([call_payoff(path, K, risk_free_rate, T) for path in share_price_path])
    # risk amount due to default risk, on each sample path
    amount_lost = np.array([np.exp(-risk_free_rate*T)*(1-recover)*(term_firm_val < D)*call_val for term_firm_val, call_val in zip(firm_value_path[:,-1], up_and_out_payoff)])
    
    # simulated quantities
    # Final share price and firm value
    sim_share_price[i-1] = np.mean(share_price_path[:, -1])
    sim_firm_value[i-1] = np.mean(firm_value_path[:, -1])
    # unadjusted call option price
    up_and_out_price[i-1] = np.mean(up_and_out_payoff)
    # CVA
    cva_estimates[i-1] = np.mean(amount_lost)
    # adjusted call option price
    call_price_adj[i-1] = up_and_out_price[i-1] - cva_estimates[i-1]
    

### Results Reporting

In [ ]:
print('--'*40)
print(" Results based on longest (50000) Monte Carlo sampling: ")
print("     Firm Value             Share Price    Up-and-Out Option Price      CVA       Adjusted Option Price")
print("  ",sim_firm_value[-1],"  ", sim_share_price[-1], "    ", up_and_out_price[-1], "   ", cva_estimates[-1], "    ", call_price_adj[-1] )
print('--'*40)

x = np.array(range(1,51))*1000
plot.plot(x, sim_share_price, label='Simulated Term Share Price')
plot.plot(x, sim_firm_value, label='Simulated Term Firm Value')
plot.xlabel("Sample Size")
plot.ylabel("Price")
plot.legend()
plot.show()

In [ ]:
plot.plot(x, up_and_out_price, label='Unadjusted Up-and-Out Call Price')
plot.plot(x, call_price_adj, label='Adjusted, Up-and-Out Call Price')
plot.plot(x, cva_estimates, label='CVA')
plot.xlabel("Sample Size")
plot.ylabel("Price")
plot.legend()
plot.show()